# GLOC Detector Dev Notes

WNixalo

2018/2/10

---

**Finetuning 1st Stage Detector**

Okay, so it looks like the Boring Detector's [detection script](https://github.com/lexfridman/boring-detector/blob/master/detect.py#L23:60) loads a model saved by the [training script](https://github.com/lexfridman/boring-detector/blob/master/detect.py#L23:60). This answers a few questions.

12:56: still not good. `weights='imagenet'` cannot be replaced by `weights='coco'` or anything like that. You need the path to the actual `.h5` file. The problem there gets back to the motivation for using the RetinaNet API in the first place. Using the `data/retinanet-model/resnet50_coco_best_v1.2.2.h5` weights path gives an error: <img src="images/modelsize.png" alt="modelsize.png"/> 

I don't know why the '9' and '720' but I know the issue is that stock RetinaNet is trying to predict multiple classes but I'm only looking for 1. And I don't know how to remove the linear / regression layers in Keras yet, and that was the whole point of using their API... but they don't provide a way to load MSCOCO weights... only ImageNet... oy.

Plan now is to try to train this anyway: upload the entire dataset to my AWS instance, and train for a few hours there with a larger batch size. Maybe it'll work?

Honestly at this point, if that doesn't work (and I have no idea why it's picking random image ids and saying they're invalid 5-item arrays... their entries are literally all the same in a CSV..) I may rip out Keras-RetinaNet and replace it with a Fastai/PyTorch-based CAM model, probably a simple ResNet or something.

**Changing Architecture**

22:50

After [a couple tweets w/ J.Howard](https://twitter.com/jeremyphoward/status/962439072739311616) I'm gonna do it. I'm going to retire the current build of the G-LOC Detector and rebuild a clean version using Fastai/PyTorch.

He suggested Pytorch SSDs, Bounding-Box Regression, and YOLO2. I really should look into those and learn. This will significantly increase the time spent on this project and push back my start on what's next, but it's worth it. The ability to ID objects and areas of interest in an image is extremely important, and I believe essential to efficient & powerful mobile (non Data Center) systems. It directly affects my next projects (actually it'd be the first thing I'd work on implementing anyway...). **Also** *it's brand new research that even J.Howard is about to start working on right now for Part II of Fastai's Deep Learning course*. So. Yeah.

WNixalo

2018/2/9

---

**Finetuning 1st Stage Detector**

RetinaNet is really proving to be a pain. Mostly because I'm not proficient in Keras and I don't want to take the time right now to be. I also don't understand why I'm creating three versions of a model using the [Boring Detector's](https://github.com/lexfridman/boring-detector/blob/master/train.py) train script. I can understand having two version of a model share memory, train & predict. But why the base 'model' too? It feels somewhat different than the stock RetinaNet I was originally using, although the training script is from RetinaNet's own API...

There's a very large overhead for initializing RetinaNet with Keras/TensorFlow too. `2745MiB`.. My machine only has `3017MiB` of GPU RAM. Simple solve to send everything to the cloud and train on my AWS p2 instance. I want to wrap up this project, not port everything over.

At bs=1: `2842MiB`. That's 97 MB / image (400x400).

Gives me an overhead of 272 MB, or enough room for a batch size of 2.

I just need it to *basically* work, nothing special. I'll try training with `bs=2` and play w/ hypars -- hopefully that'll give good results.

If not... oy. My techie side wants to swap out Keras/TF and put in a Fastai/PT CAM-based detector. My exec side wants me to get this project done **now** and get started on What's next.

<img src="images/retinanet-traintime.png" alt="retinanet-traintime.png"/>

Okay.. this is not good.

<img src="images/retinanet-wronglabels.png" alt="retinanet-wronglabels.png"/>

This is also bad. I formatted the interstage labels exactly as RetinaNet wanted them -- as far as I could tell. Why would some of the labels be invalid (a 5-item array instaed of 4) while others are fine? They were all made the same..


WNixalo

2018/2/8

---

I thought it'd be a good idea to have a notebook of thoughts/ideas/notes as I work on my projects. May help to keep my head in order (idea perhaps for future teams?) and also help with documentation for post-project writeups.. Hmm.

**Finetuning 1st Stage Detector**

CAMs Heatmaps FastAI replacing Keras/RetinaNet if I decide to make further major changes to the first stage detector.

Keras learning-rate is part of the optimizer attribute of the model? Doesn't seem immediately accessible: `model.optimizer.lr` returns: <img src="images/keras-λr.png" alt="keras-λr.png"/>